In [10]:
from deepsea import *
from keras.models import load_model

In [11]:
model = load_model("weights-improvement-09-0.06-full.hdf5")

In [ ]:
valid_X, valid_Y, train_X, train_Y, test_X, test_Y = loaddata()

# Trying out DeepLift for explanations

In [2]:
import deeplift
from deeplift.conversion import keras_conversion as kc